# 第 4 阶段：DeepAgents 深度研究

本 Notebook 用于学习和调试第 4 阶段的核心功能：
- DeepAgent 深度研究架构
- 多代理协作（搜索、分析、总结）
- 网络搜索集成
- 文档分析集成
- 文件系统管理
- 研究报告生成

**参考文档**: `docs/stage_04/`（计划中）  
**参考脚本**: `scripts/test_deep_research.py`

## 1. 环境准备

In [ ]:
# 添加项目根目录到 Python 路径
import sys
from pathlib import Path

backend_dir = Path.cwd()
if backend_dir.name == 'notebooks':
    backend_dir = backend_dir.parent

if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

print(f"✅ 项目根目录: {backend_dir}")

# 导入必要的模块
import time
import uuid
from config import settings, get_logger

logger = get_logger(__name__)

## 2. 检查配置

In [ ]:
print("\n配置检查")
print("="*60)
print(f"OpenAI API: {'✅ 已配置' if settings.openai_api_key else '❌ 未配置'}")
print(f"Tavily API: {'✅ 已配置' if settings.tavily_api_key else '❌ 未配置'}")
print(f"模型: {settings.openai_model}")
print(f"数据目录: {settings.DATA_DIR}")
print("="*60)

if not settings.tavily_api_key:
    print("\n⚠️  警告: 未配置 TAVILY_API_KEY，网络搜索功能将不可用")
    print("   请在 .env 文件中设置 TAVILY_API_KEY")

## 3. 文件系统功能

### 3.1 创建文件系统

In [ ]:
from core.tools.filesystem import get_filesystem

# 创建文件系统实例
thread_id = f"notebook_fs_{uuid.uuid4().hex[:8]}"
fs = get_filesystem(thread_id)

print(f"✅ 文件系统创建成功")
print(f"   Thread ID: {thread_id}")
print(f"   工作空间: {fs.workspace_path}")

### 3.2 写入文件

In [ ]:
# 写入测试文件
test_content = """# 研究笔记

## 主题
LangChain 1.0 新特性研究

## 关键发现
1. 改进的 Agent 架构
2. 更好的流式输出支持
3. 增强的工具调用能力

## 参考资料
- LangChain 官方文档
- GitHub Release Notes
"""

# 写入到 notes 子目录
fs.write_file(
    filename="research_note.md",
    content=test_content,
    subdirectory="notes"
)

print("✅ 文件写入成功: notes/research_note.md")

### 3.3 读取文件

In [ ]:
# 读取文件
content = fs.read_file("research_note.md", subdirectory="notes")

print("📄 文件内容:")
print("="*60)
print(content)
print("="*60)

### 3.4 列出文件

In [ ]:
# 列出所有文件
files = fs.list_files()

print(f"\n📚 工作空间文件列表 ({len(files)} 个):")
for f in files:
    print(f"  - {f}")

### 3.5 搜索文件

In [ ]:
# 搜索包含特定关键词的文件
search_results = fs.search_files("LangChain")

print(f"\n🔍 搜索结果 (关键词: 'LangChain'):")
print(f"找到 {len(search_results)} 个匹配文件:")
for result in search_results:
    print(f"  - {result}")

### 3.6 删除文件

In [ ]:
# 删除测试文件
# fs.delete_file("research_note.md", subdirectory="notes")
# print("✅ 文件已删除")

print("⚠️  取消注释上面的代码以删除文件")

## 4. DeepAgent 基础研究

### 4.1 创建 DeepAgent（仅网络搜索）

In [ ]:
from deep_research import create_deep_research_agent

# 检查 Tavily API Key
if not settings.tavily_api_key:
    print("❌ 未配置 TAVILY_API_KEY，无法使用网络搜索功能")
    print("   请在 .env 文件中设置 TAVILY_API_KEY")
else:
    # 创建 DeepAgent
    research_thread_id = f"research_{uuid.uuid4().hex[:8]}"
    
    agent = create_deep_research_agent(
        thread_id=research_thread_id,
        enable_web_search=True,
        enable_doc_analysis=False,
    )
    
    print(f"✅ DeepAgent 创建成功")
    print(f"   Thread ID: {research_thread_id}")
    print(f"   网络搜索: 启用")
    print(f"   文档分析: 禁用")

### 4.2 执行基础研究

In [ ]:
# 定义研究问题
query = "LangChain 1.0 有哪些主要新特性？"

print(f"\n🔬 研究问题: {query}")
print("\n⏳ 正在执行研究任务，这可能需要几分钟...\n")

if settings.tavily_api_key:
    start_time = time.time()
    
    # 执行研究
    result = agent.research(query)
    
    elapsed_time = time.time() - start_time
    
    print(f"\n✅ 研究完成！耗时: {elapsed_time:.1f} 秒\n")
else:
    print("⚠️  跳过研究（需要 Tavily API Key）")

### 4.3 查看研究计划

In [ ]:
if settings.tavily_api_key and result.get("plan"):
    plan = result["plan"]
    
    print("\n📋 研究计划")
    print("="*60)
    print(f"研究目标: {plan.get('research_goal', 'N/A')}")
    
    if plan.get('search_keywords'):
        print(f"\n搜索关键词:")
        for kw in plan['search_keywords']:
            print(f"  - {kw}")
    
    if plan.get('analysis_steps'):
        print(f"\n分析步骤:")
        for i, step in enumerate(plan['analysis_steps'], 1):
            print(f"  {i}. {step}")
    
    print("="*60)
else:
    print("⚠️  未获取到研究计划")

### 4.4 查看完成的步骤

In [ ]:
if settings.tavily_api_key and result.get("steps_completed"):
    steps = result["steps_completed"]
    
    print("\n✅ 完成的步骤")
    print("="*60)
    for step, completed in steps.items():
        status = "✅" if completed else "❌"
        print(f"{status} {step}")
    print("="*60)
else:
    print("⚠️  未获取到步骤信息")

### 4.5 查看最终报告

In [ ]:
if settings.tavily_api_key and result.get("final_report"):
    report = result["final_report"]
    
    print("\n📊 最终研究报告")
    print("="*60)
    print(report)
    print("="*60)
else:
    print("⚠️  未生成最终报告")

### 4.6 查看生成的文件

In [ ]:
if settings.tavily_api_key:
    # 获取研究工作空间的文件系统
    research_fs = get_filesystem(research_thread_id)
    files = research_fs.list_files()
    
    print(f"\n📁 生成的文件 ({len(files)} 个):")
    print("="*60)
    for f in files:
        print(f"  📄 {f}")
    print("="*60)
else:
    print("⚠️  跳过文件查看")

## 5. DeepAgent 完整研究（网络 + 文档）

### 5.1 准备文档索引

In [ ]:
# 检查是否有可用的索引
from pathlib import Path

index_path = Path(settings.vector_store_path) / "test_index"

if index_path.exists():
    print(f"✅ 找到测试索引: {index_path}")
    has_index = True
else:
    print(f"⚠️  未找到测试索引: {index_path}")
    print("   请先运行 RAG 索引构建:")
    print("   python scripts/update_index.py --collection test_index --path data/documents/test")
    has_index = False

### 5.2 加载文档检索器

In [ ]:
if has_index:
    from rag import get_embeddings, load_vector_store, create_retriever_tool
    
    print("🔄 加载文档索引...")
    
    # 加载 embedding 和向量存储
    embeddings = get_embeddings()
    vector_store = load_vector_store(str(index_path), embeddings)
    
    # 创建检索器工具
    retriever = vector_store.as_retriever()
    retriever_tool = create_retriever_tool(retriever)
    
    print("✅ 文档索引加载成功")
else:
    print("⚠️  跳过文档加载")
    retriever_tool = None

### 5.3 创建完整 DeepAgent

In [ ]:
if has_index and settings.tavily_api_key:
    # 创建完整的 DeepAgent（网络搜索 + 文档分析）
    full_thread_id = f"full_research_{uuid.uuid4().hex[:8]}"
    
    full_agent = create_deep_research_agent(
        thread_id=full_thread_id,
        enable_web_search=True,
        enable_doc_analysis=True,
        retriever_tool=retriever_tool,
    )
    
    print(f"✅ 完整 DeepAgent 创建成功")
    print(f"   Thread ID: {full_thread_id}")
    print(f"   网络搜索: 启用")
    print(f"   文档分析: 启用")
else:
    print("⚠️  跳过完整 DeepAgent 创建（需要索引和 Tavily API Key）")

### 5.4 执行完整研究

In [ ]:
if has_index and settings.tavily_api_key:
    # 定义研究问题
    full_query = "什么是 RAG？它有哪些应用场景？"
    
    print(f"\n🔬 研究问题: {full_query}")
    print("\n⏳ 正在执行完整研究任务（网络 + 文档），这可能需要更长时间...\n")
    
    start_time = time.time()
    
    # 执行研究
    full_result = full_agent.research(full_query)
    
    elapsed_time = time.time() - start_time
    
    print(f"\n✅ 完整研究完成！耗时: {elapsed_time:.1f} 秒\n")
else:
    print("⚠️  跳过完整研究")

### 5.5 查看完整研究结果

In [ ]:
if has_index and settings.tavily_api_key:
    # 显示完成的步骤
    print("\n✅ 完成的步骤")
    print("="*60)
    steps = full_result.get("steps_completed", {})
    for step, completed in steps.items():
        status = "✅ 完成" if completed else "❌ 未完成"
        print(f"{status} - {step}")
    print("="*60)
    
    # 显示最终报告
    if full_result.get("final_report"):
        print("\n📊 最终研究报告")
        print("="*60)
        print(full_result["final_report"])
        print("="*60)
    
    # 显示生成的文件
    full_fs = get_filesystem(full_thread_id)
    files = full_fs.list_files()
    
    print(f"\n📁 生成的文件 ({len(files)} 个):")
    for f in files:
        print(f"  📄 {f}")
else:
    print("⚠️  跳过结果查看")

## 6. 研究结果分析

### 6.1 读取研究报告文件

In [ ]:
if settings.tavily_api_key:
    # 读取最终报告文件
    research_fs = get_filesystem(research_thread_id)
    
    try:
        # 尝试读取最终报告
        report_content = research_fs.read_file("final_report.md")
        
        print("\n📄 最终报告文件内容")
        print("="*60)
        print(report_content)
        print("="*60)
    except Exception as e:
        print(f"⚠️  无法读取报告文件: {e}")
else:
    print("⚠️  跳过报告读取")

### 6.2 分析研究数据

In [ ]:
if settings.tavily_api_key and result:
    print("\n📈 研究数据分析")
    print("="*60)
    
    # 统计信息
    print(f"研究问题: {query}")
    print(f"耗时: {elapsed_time:.1f} 秒")
    
    # 步骤完成情况
    steps = result.get("steps_completed", {})
    completed_count = sum(1 for v in steps.values() if v)
    total_count = len(steps)
    
    print(f"\n步骤完成: {completed_count}/{total_count}")
    
    # 文件生成情况
    research_fs = get_filesystem(research_thread_id)
    files = research_fs.list_files()
    
    print(f"生成文件数: {len(files)}")
    
    # 报告长度
    if result.get("final_report"):
        report_length = len(result["final_report"])
        print(f"报告长度: {report_length} 字符")
    
    print("="*60)
else:
    print("⚠️  跳过数据分析")

## 7. 自定义研究任务

In [ ]:
def run_custom_research(query: str, enable_web: bool = True, enable_doc: bool = False):
    """运行自定义研究任务"""
    
    if not settings.tavily_api_key and enable_web:
        print("❌ 需要 Tavily API Key 才能使用网络搜索")
        return None
    
    # 创建 thread_id
    custom_thread_id = f"custom_{uuid.uuid4().hex[:8]}"
    
    print(f"\n{'='*60}")
    print(f"自定义研究任务")
    print(f"{'='*60}")
    print(f"Thread ID: {custom_thread_id}")
    print(f"问题: {query}")
    print(f"网络搜索: {'启用' if enable_web else '禁用'}")
    print(f"文档分析: {'启用' if enable_doc else '禁用'}")
    print(f"{'='*60}\n")
    
    # 创建 agent
    agent = create_deep_research_agent(
        thread_id=custom_thread_id,
        enable_web_search=enable_web,
        enable_doc_analysis=enable_doc,
        retriever_tool=retriever_tool if enable_doc and has_index else None,
    )
    
    print("⏳ 研究中...\n")
    
    # 执行研究
    start = time.time()
    result = agent.research(query)
    elapsed = time.time() - start
    
    print(f"\n✅ 研究完成！耗时: {elapsed:.1f} 秒\n")
    
    # 显示结果
    if result.get("final_report"):
        print("📊 研究报告:")
        print("="*60)
        print(result["final_report"][:500] + "...")
        print("="*60)
    
    return result

# 使用示例（取消注释以运行）
# custom_result = run_custom_research(
#     query="什么是 Transformer 架构？",
#     enable_web=True,
#     enable_doc=False
# )

## 8. API 集成示例

### 8.1 API 端点说明

In [ ]:
print("""
DeepAgent API 端点:

1. 启动研究任务:
   POST /deep-research/start
   Body: {
     "query": "研究问题",
     "enable_web_search": true,
     "enable_doc_analysis": false
   }

2. 查询任务状态:
   GET /deep-research/status/{thread_id}

3. 获取研究结果:
   GET /deep-research/result/{thread_id}

4. 列出生成的文件:
   GET /deep-research/files/{thread_id}

5. 读取特定文件:
   GET /deep-research/file/{thread_id}/{filename}
""")

### 8.2 使用 requests 调用 API（需要启动服务器）

In [ ]:
# 示例代码（需要先启动 API 服务器）
print("""
使用 requests 调用 API 的示例代码:

import requests

# 1. 启动研究
response = requests.post(
    "http://localhost:8000/deep-research/start",
    json={
        "query": "LangChain 1.0 新特性",
        "enable_web_search": True,
        "enable_doc_analysis": False
    }
)
thread_id = response.json()["thread_id"]

# 2. 查询状态
status = requests.get(f"http://localhost:8000/deep-research/status/{thread_id}")
print(status.json())

# 3. 获取结果
result = requests.get(f"http://localhost:8000/deep-research/result/{thread_id}")
print(result.json())
""")

## 9. 总结

通过本 Notebook，你已经学习了：

✅ DeepAgent 深度研究架构  
✅ 文件系统管理（读写、搜索、删除）  
✅ 基础研究（网络搜索）  
✅ 完整研究（网络 + 文档分析）  
✅ 研究计划生成  
✅ 多步骤研究流程  
✅ 研究报告生成  
✅ API 集成方式

**下一步**: 学习第 5 阶段 - Guardrails 安全防护（`stage_05_guardrails.ipynb`）